In [ ]:
# | default_exp wb_veg

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.utils import *

In [1]:
# | export

import warnings
import collections
import numpy as np
from typing import Dict
from pandera.typing import DataFrame
from sureau_ecos_py.climate_utils import (
    day_length,
    compute_pet,
    potential_par,
    rg_units_conversion,
    compute_vpd_from_t_rh,
    calculate_rh_diurnal_pattern,
    rg_watt_ppfd_umol_conversions,
    calculate_radiation_diurnal_pattern,
    calculate_temperature_diurnal_pattern,
)
from sureau_ecos_py.create_vegetation_parameters import (
    create_vegetation_parameters
)
from sureau_ecos_py.create_climate_data import create_climate_data
from sureau_ecos_py.create_modeling_options import create_modeling_options
from sureau_ecos_py.create_soil_parameters import create_soil_parameters
from sureau_ecos_py.create_stand_parameters import create_stand_parameters

In [12]:
# | export
def new_wb_veg(veg_params:Dict # Dictionary created using the `create_vegetation_parameters` functions
) -> Dict:

    "Create an object wb_veg from veg_params"

    # Assert parameters ---------------------------------------------------------
    assert (
        isinstance(veg_params, Dict)
    ), f"veg_param must be a Dictionary not a {type(veg_params)}"


    # Add params to wb_veg dictionary -------------------------------------------

    # Create Empty dict
    wb_veg = collections.defaultdict(list)

    # Add veg_params
    wb_veg["params"] = veg_params

    # Add Stem and Leaf turgor point
    wb_veg['params']['psi_tlp_leaf'] = wb_veg["params"]["pifullturgor_leaf"]*wb_veg["params"]["epsilonsym_leaf"]/(wb_veg["params"]["pifullturgor_leaf"]+wb_veg["params"]["epsilonsym_leaf"])
    wb_veg['params']['psi_tlp_stem'] = wb_veg["params"]["pifullturgor_stem"]*wb_veg["params"]["epsilonsym_stem"]/(wb_veg["params"]["pifullturgor_stem"]+wb_veg["params"]["epsilonsym_stem"])

    # Add potentials
    wb_veg['psi_lapo'] = 0
    wb_veg['psi_sapo'] = 0
    wb_veg['psi_lsym'] = 0
    wb_veg['psi_ssym'] = 0

    # FP replaced "mem" by "cav" (when cavitation starts)
    wb_veg['psi_lapo_cav'] = 0
    wb_veg['psi_sapo_cav'] = 0
    wb_veg['psi_all_soil'] = 0

    # Conductance & capacitance (mmol/m2/s/MPa) Here on leaf area basis but they
    # are to be updated as a function symplasm conductance and leaf area hydraulic
    # conductances

    # constant value during simulation
    wb_veg["k_plant"] =  wb_veg["params"]["k_plant_init"]

    # constant value during simulation
    wb_veg["k_lsym"]  =  wb_veg["params"]["k_lsym_init"]

    # constant value during simulation
    #wb_veg["k_ssym"]  =  wb_veg["params"]["k_ssym_init"]


    return wb_veg


#### __Example: __

In [3]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="sigmoid",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="campbell",
)

In [4]:
stand_params = create_stand_parameters(
    file_path="./sample_data/stand_example.csv",
    lai_max=None,
    latitude=None,
    longitude=None,
    sep=";",
)

In [5]:
soil_params = create_soil_parameters(
    file_path="./sample_data/soil_example.csv",
    modeling_options=modeling_options_dict,
    default_soil=False,
    offset_psoil=1,
    psoil_at_field_capacity=-1,
)

There is an offset on Psoil of 1 MPa
Psoil at field capacity = -0.001 MPa
You are using campbell pedotransfer formulation
Available water capacity Wilting: 273.74477742718403 mm
Available water capacity Residual: 307.81195119276424 mm
Can soil_params["v_soil_storage_capacity"] be negative?? Ask


In [14]:
vegetation_params = create_vegetation_parameters(
    file_path="./sample_data/vegetation_example_wide.csv",
    list_of_parameters=None,
    soil_parameters=soil_params,
    stand_parameters=stand_params,
    modeling_options=modeling_options_dict,
    sep=",",
)
vegetation_params

frac_leaf_sym' set to 0.4
Available water capacity @Tlp (Campbell):280.7301527422195 mm
Available water capacity @P50 (Campbell):283.4407203700769 mm


/workspaces/sureau_ecos_py/sureau_ecos_py/plant_utils.py:899: UserWarning: Foliage evergreen has no params
  warnings.warn("Foliage evergreen has no params")
/workspaces/sureau_ecos_py/sureau_ecos_py/create_vegetation_parameters.py:106: UserWarning: Make sure that depth from soil_parameters is in meters
  warnings.warn("Make sure that depth from soil_parameters is in meters")


defaultdict(list,
            {'c_lapoinit': 1e-05,
             'c_sapoinit': 2e-05,
             'fcrit': 450,
             'foliage': 'evergreen',
             'jarvis_par': 0.006,
             'k': 0.5,
             'k_plantinit': 0.62,
             'k_ssyminit': 0.26,
             'ldmc': 570,
             'lma': 106,
             'p12_gs': -2.07,
             'p50_vc_leaf': -3.4,
             'p50_vc_stem': -3.4,
             'p88_gs': -2.62,
             'pt_coeff': 1.14,
             'pifullturgor_leaf': -2.1,
             'pifullturgor_stem': -2.1,
             'psi_close': -2,
             'psi_start_closing': -0.5,
             'q10_1_gmin': 1.2,
             'q10_2_gmin': 4.8,
             'tphase_gmin': 37.5,
             'tbase': 3,
             'tgs_optim': 25,
             'tgs_sens': 17,
             'apofrac_leaf': 0.4,
             'apofrac_stem': 0.4,
             'betarootprofile': 0.97,
             'canopystorageparam': 1.5,
             'daystart': 55,
         

In [13]:
new_wb_veg(vegetation_params)

defaultdict(list,
            {'params': defaultdict(list,
                         {'c_lapoinit': 1e-05,
                          'c_sapoinit': 2e-05,
                          'fcrit': 450,
                          'foliage': 'evergreen',
                          'jarvis_par': 0.006,
                          'k': 0.5,
                          'k_plantinit': 0.62,
                          'k_ssyminit': 0.26,
                          'ldmc': 570,
                          'lma': 106,
                          'p12_gs': -2.07,
                          'p50_vc_leaf': -3.4,
                          'p50_vc_stem': -3.4,
                          'p88_gs': -2.62,
                          'pt_coeff': 1.14,
                          'pifullturgor_leaf': -2.1,
                          'pifullturgor_stem': -2.1,
                          'psi_close': -2,
                          'psi_start_closing': -0.5,
                          'q10_1_gmin': 1.2,
                          'q10_2_